In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "QTCON"
to_date = '2021-06-01'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']


    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit
0,CRIX.UPBIT.KRW-QTCON,2021-06-10T14:00:00+00:00,2021-06-10T23:00:00+09:00,68.6,68.8,66.4,66.8,7.388177e+07,4.976362e+09,1623334362731,30
1,CRIX.UPBIT.KRW-QTCON,2021-06-10T13:30:00+00:00,2021-06-10T22:30:00+09:00,69.9,69.9,67.6,68.6,1.819486e+08,1.247616e+10,1623333599789,30
2,CRIX.UPBIT.KRW-QTCON,2021-06-10T13:00:00+00:00,2021-06-10T22:00:00+09:00,71.3,71.6,69.7,69.9,9.598321e+07,6.768288e+09,1623331800350,30
3,CRIX.UPBIT.KRW-QTCON,2021-06-10T12:30:00+00:00,2021-06-10T21:30:00+09:00,72.3,72.8,71.2,71.3,1.203125e+08,8.648812e+09,1623329999487,30
4,CRIX.UPBIT.KRW-QTCON,2021-06-10T12:00:00+00:00,2021-06-10T21:00:00+09:00,72.2,72.7,70.8,72.3,1.099278e+08,7.872711e+09,1623328200057,30
...,...,...,...,...,...,...,...,...,...,...,...
471,CRIX.UPBIT.KRW-QTCON,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,52.6,52.7,52.3,52.4,7.925233e+05,4.158999e+07,1622482168004,30
472,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,52.4,52.7,52.3,52.6,6.720946e+05,3.527668e+07,1622480397914,30
473,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,52.4,52.8,52.2,52.4,1.788808e+06,9.389585e+07,1622478547916,30
474,CRIX.UPBIT.KRW-QTCON,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,52.5,52.7,52.2,52.5,4.414198e+05,2.315340e+07,1622476737626,30


In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
df['Date'][0] = df['candleDateTimeKst'][0].split('T')
df['Date'][0][0]


'2021-06-10'

In [5]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [6]:
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit,Date,hour,minute
0,CRIX.UPBIT.KRW-QTCON,2021-06-10T14:00:00+00:00,2021-06-10T23:00:00+09:00,68.6,68.8,66.4,66.8,7.388177e+07,4.976362e+09,1623334362731,30,2021-06-10,23,0
1,CRIX.UPBIT.KRW-QTCON,2021-06-10T13:30:00+00:00,2021-06-10T22:30:00+09:00,69.9,69.9,67.6,68.6,1.819486e+08,1.247616e+10,1623333599789,30,2021-06-10,22,30
2,CRIX.UPBIT.KRW-QTCON,2021-06-10T13:00:00+00:00,2021-06-10T22:00:00+09:00,71.3,71.6,69.7,69.9,9.598321e+07,6.768288e+09,1623331800350,30,2021-06-10,22,0
3,CRIX.UPBIT.KRW-QTCON,2021-06-10T12:30:00+00:00,2021-06-10T21:30:00+09:00,72.3,72.8,71.2,71.3,1.203125e+08,8.648812e+09,1623329999487,30,2021-06-10,21,30
4,CRIX.UPBIT.KRW-QTCON,2021-06-10T12:00:00+00:00,2021-06-10T21:00:00+09:00,72.2,72.7,70.8,72.3,1.099278e+08,7.872711e+09,1623328200057,30,2021-06-10,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,CRIX.UPBIT.KRW-QTCON,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,52.6,52.7,52.3,52.4,7.925233e+05,4.158999e+07,1622482168004,30,2021-06-01,02,0
472,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,52.4,52.7,52.3,52.6,6.720946e+05,3.527668e+07,1622480397914,30,2021-06-01,01,30
473,CRIX.UPBIT.KRW-QTCON,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,52.4,52.8,52.2,52.4,1.788808e+06,9.389585e+07,1622478547916,30,2021-06-01,01,0
474,CRIX.UPBIT.KRW-QTCON,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,52.5,52.7,52.2,52.5,4.414198e+05,2.315340e+07,1622476737626,30,2021-06-01,00,30


In [7]:

drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,23,0,66.8,68.8,66.4,7.388177e+07
1,22,30,68.6,69.9,67.6,1.819486e+08
2,22,0,69.9,71.6,69.7,9.598321e+07
3,21,30,71.3,72.8,71.2,1.203125e+08
4,21,0,72.3,72.7,70.8,1.099278e+08


In [8]:
df['tradePrice'][0]

66.8

In [18]:
df['rsi_ud'] = 0

for j in range(2):
    upper = 0
    down = 0
    for i in range(14):
        if df['tradePrice'][i+j] > df['tradePrice'][i+j+1]:
            upper += df['tradePrice'][i+j] - df['tradePrice'][i+j+1]
            # print(upper)
        elif df['tradePrice'][i+j] < df['tradePrice'][i+j+1]:
            down += df['tradePrice'][i+j+1] - df['tradePrice'][i+j]
            # print(down)
        else:
            continue
    print(upper,down)
    AU = upper/14
    AD = down/14
    RS = AU / AD
    print(RS)
    RSI = (RS/(1+RS))
    print(RSI)

10.700000000000017 17.100000000000023
0.6257309941520469
0.3848920863309353
10.700000000000017 16.300000000000026
0.656441717791411
0.3962962962962963


In [15]:
x = 54.0225/100
a= x/(1-x)
a

1.1749768908705343

In [ ]:
10.7
-16.3